# Date Night!

A play in three acts... perhaps four.

## Part 0: Install and import libraries

In [1]:
# For making HTTP requests
#!pip3 install requests
import requests

# Install BeautifulSoup
#!pip3 install beautifulsoup4

# Install additional BS parsers
#!pip3 install lxml html5lib

# Finally, import it
from bs4 import BeautifulSoup

import pandas as pd
import numpy as np

#!pip3 install sklearn
from sklearn.cluster import KMeans

#!pip3 install geopy
from geopy.geocoders import Nominatim

#!pip3 install folium
#!pip3 install matplotlib
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium

## Part 1: User Region, Range, and Proximity

We prime our analysis by setting user data.

In [2]:
user_region_str = "Seattle, WA"
geolocator = Nominatim(user_agent="to_explorer")
user_region_geo = geolocator.geocode(user_region_str)

user_range_m = 1000
user_proximity_m = 75

## Part 2: Get Entertainment Venues

In [3]:
# Explore the areas using Foursquare API

#CLIENT_ID = '0Z0XSTUX1JNVS1PZXKK3RD3EYDTSVUKYQVVB4BAP4CAQSNET' # your Foursquare ID
#CLIENT_SECRET = 'WDCWJJUFALFIPVBIXJCQCC1INMBYH03GBNALELPZOH3DXHRQ' # your Foursquare Secret
CLIENT_ID = 'OSN2GUSX2A3XKXANKFVTXA0MMCS523YEU00IBNNNLKWDR34P' # your Foursquare ID
CLIENT_SECRET = 'GJ0LWQ2HXJJVZKTD1GIJ5F030C40VPZHUVR1XTQHUM1SLGC3' # your Foursquare Secret

VERSION = '20180605' # Foursquare API version
LIMIT = 100

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: OSN2GUSX2A3XKXANKFVTXA0MMCS523YEU00IBNNNLKWDR34P
CLIENT_SECRET:GJ0LWQ2HXJJVZKTD1GIJ5F030C40VPZHUVR1XTQHUM1SLGC3


Define Foursquare category IDs

In [4]:
ENTERTAINMENT_ID = '4d4b7104d754a06370d81259'
RESTAURANT_ID = '4d4b7105d754a06374d81259'
BAR_ID = '4d4b7105d754a06376d81259'

Define a function to query Foursquare for venues within a radius of a given location and filtered for a category ID

In [5]:
def getNearbyVenues(names, latitudes, longitudes, categoryID, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
          
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&categoryId={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius,
            categoryID,
            LIMIT)
            
        # make the GET request
        response = requests.get(url).json()
        if ("groups" in response["response"]):
            # we have a well formed response, so continue
            results = response["response"]['groups'][0]['items']
        
            # return only relevant information for each nearby venue
            venues_list.append([(
                name, 
                v['venue']['name'],
                v['venue']['id'],
                v['venue']['location']['lat'], 
                v['venue']['location']['lng'],  
                v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    if (len(nearby_venues.index) > 0): 
        nearby_venues.columns = ['Region', 
                  'Venue Name', 
                  'Venue ID',
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Now let's query Foursquare for each neighborhood's venues.

In [6]:
entertainment_venues = getNearbyVenues( names=[user_region_str],
                                        latitudes=[user_region_geo.latitude],
                                        longitudes=[user_region_geo.longitude],
                                        categoryID=ENTERTAINMENT_ID,
                                        radius=user_range_m
                               )

Let's check the size of the resulting dataframe

In [7]:
entertainment_venues.sort_values(['Venue Category'], ascending = True, axis = 0, inplace = True)
entertainment_venues

,Region,Venue Name,Venue ID,Venue Latitude,Venue Longitude,Venue Category
33,"Seattle, WA",Bryan Ohno Gallery,4b8c3445f964a520f4c332e3,47.599911,-122.326752,Art Gallery
34,"Seattle, WA",Linda Hodges Gallery,4b8c305df964a5201fc332e3,47.599459,-122.333795,Art Gallery
65,"Seattle, WA",Modernist Cuisine Gallery,5c033f14e4c459002cd9c82e,47.608116,-122.339824,Art Gallery
39,"Seattle, WA",Tashiro Kaplan Bldg,4ac55be8f964a520b2b120e3,47.601113,-122.329778,Art Gallery
29,"Seattle, WA",JDRF Gingerbread Village,4cf16e377e0da1cde1949197,47.610838,-122.334255,Art Gallery
...,...,...,...,...,...,...
21,"Seattle, WA",Market Ghost Tours,4bb6bcacf562ef3b33b03097,47.608889,-122.340302,Tour Provider
62,"Seattle, WA",Chinatown Discovery Tours,4e680c4bb993f95a3f69f089,47.598615,-122.322958,Tour Provider
51,"Seattle, WA",Seattle Cycling Tours,4e03c7db45ddb46455771f12,47.612181,-122.333107,Tour Provider
6,"Seattle, WA",Bill Speidel's Underground Tour,4a88a98df964a520500720e3,47.602151,-122.333980,Tour Provider


Let's check how many venues were returned for each neighborhood

In [8]:
entertainment_venues.groupby('Venue Category').count()

,Region,Venue Name,Venue ID,Venue Latitude,Venue Longitude
Venue Category,,,,,
Art Gallery,24,24,24,24,24
Art Museum,3,3,3,3,3
Comedy Club,3,3,3,3,3
Concert Hall,2,2,2,2,2
Dance Studio,1,1,1,1,1
History Museum,2,2,2,2,2
Indie Theater,1,1,1,1,1
Jazz Club,1,1,1,1,1
Mini Golf,1,1,1,1,1


Let's find out how many unique categories can be curated from all the returned venues

In [9]:
print('There are {} unique categories.'.format(len(entertainment_venues['Venue Category'].unique())))

There are 18 unique categories.


Finally, let's make a plot of our results just because we can

In [10]:
# create map
map_entertainment_venues = folium.Map(location=[user_region_geo.latitude, user_region_geo.longitude], zoom_start=13)

# set color scheme for the clusters
#x = np.arange(kclusters)
#ys = [i + x + (i*x)**2 for i in range(kclusters)]
#colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
#rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for venue, v_id, lat, lon in zip(entertainment_venues['Venue Name'], entertainment_venues['Venue ID'], entertainment_venues['Venue Latitude'], entertainment_venues['Venue Longitude']):
    label = folium.Popup("<a target='_blank' href='https://foursquare.com/v/" + str(v_id) + "'>" + str(venue) + "</a>")
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        #color=rainbow[cluster-1],
        fill=True,
        #fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_entertainment_venues)
       
map_entertainment_venues

## Part 3: Find Date Triples

For each entertainment venue, we'll query Foursquare for separate lists of Restaurants and Bars within our proximity, then compile a complete list of all possible date triples.

In [11]:
column_names = ['e_name', 'e_id', 'e_cat', 'e_lat', 'e_lng', 'r_name', 'r_id', 'r_cat', 'r_lat', 'r_lng', 'b_name', 'b_id', 'b_cat', 'b_lat', 'b_lng']
date_triples = pd.DataFrame(columns = column_names)
for name,v_id,cat,lat,lng in zip(entertainment_venues['Venue Name'],entertainment_venues['Venue ID'],entertainment_venues['Venue Category'], entertainment_venues['Venue Latitude'],entertainment_venues['Venue Longitude']):
    print("Looking for triples near: " + name)
    bars = getNearbyVenues(names=[name],
                            latitudes=[lat],
                            longitudes=[lng],
                            categoryID=BAR_ID,
                            radius=user_proximity_m )
    rests = getNearbyVenues(names=[name],
                            latitudes=[lat],
                            longitudes=[lng],
                            categoryID=RESTAURANT_ID,
                            radius=user_proximity_m )
    if (len(bars.index) > 0) and (len(rests.index) > 0):
        for r_ind, r in rests.iterrows():
            for b_ind, b in bars.iterrows():
                triple = pd.DataFrame([[name, v_id, cat, lat, lng, 
                                       r['Venue Name'], r['Venue ID'], r['Venue Category'], r['Venue Latitude'], r['Venue Longitude'],
                                       b['Venue Name'], b['Venue ID'], b['Venue Category'], b['Venue Latitude'], b['Venue Longitude'] ]],
                                     columns=column_names)
                date_triples = date_triples.append(triple, ignore_index=True)

date_triples

Looking for triples near: Bryan Ohno Gallery
Looking for triples near: Linda Hodges Gallery
Looking for triples near: Modernist Cuisine Gallery
Looking for triples near: Tashiro Kaplan Bldg
Looking for triples near: JDRF Gingerbread Village
Looking for triples near: OK Hotel
Looking for triples near: Pike Street Press
Looking for triples near: SOIL Gallery
Looking for triples near: Greg Kucera Gallery
Looking for triples near: Pioneer Square Art Walk
Looking for triples near: OO studios
Looking for triples near: Seattle Totem Pole
Looking for triples near: SAM Gallery
Looking for triples near: Glasshouse Studio
Looking for triples near: Catherine Person Gallery
Looking for triples near: METHOD Gallery
Looking for triples near: JMost Photography
Looking for triples near: William Traver Gallery
Looking for triples near: Davidson Galleries
Looking for triples near: KOBO
Looking for triples near: R E Welch Gallery
Looking for triples near: Azuma Gallery
Looking for triples near: Gallery IM

,e_name,e_id,e_cat,e_lat,e_lng,r_name,r_id,r_cat,r_lat,r_lng,b_name,b_id,b_cat,b_lat,b_lng
0,Linda Hodges Gallery,4b8c305df964a5201fc332e3,Art Gallery,47.599459,-122.333795,General Porpoise Coffee & Doughnuts,5aabfe234940bc48fece464f,Donut Shop,47.599113,-122.334327,The Pharmacy,53643967498eff6b63587c78,Lounge,47.599275,-122.333053
1,Linda Hodges Gallery,4b8c305df964a5201fc332e3,Art Gallery,47.599459,-122.333795,General Porpoise Coffee & Doughnuts,5aabfe234940bc48fece464f,Donut Shop,47.599113,-122.334327,Temple Billiards,4ace4937f964a520a3cf20e3,Pool Hall,47.599286,-122.333223
2,Linda Hodges Gallery,4b8c305df964a5201fc332e3,Art Gallery,47.599459,-122.333795,General Porpoise Coffee & Doughnuts,5aabfe234940bc48fece464f,Donut Shop,47.599113,-122.334327,Aura Nightclub,4b1b6217f964a52027fb23e3,Nightclub,47.599643,-122.334266
3,Linda Hodges Gallery,4b8c305df964a5201fc332e3,Art Gallery,47.599459,-122.333795,General Porpoise Coffee & Doughnuts,5aabfe234940bc48fece464f,Donut Shop,47.599113,-122.334327,Joseph NYLA,4d9aa723674ca1436993c043,Nightlife Spot,47.599621,-122.334534
4,Linda Hodges Gallery,4b8c305df964a5201fc332e3,Art Gallery,47.599459,-122.333795,General Porpoise Coffee & Doughnuts,5aabfe234940bc48fece464f,Donut Shop,47.599113,-122.334327,The Kitchen,50f24a34e4b0c27bdc10db23,Cocktail Bar,47.599677,-122.334565
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1272,Eat Seattle,5591a095498e4f08a11b995f,Tour Provider,47.608695,-122.339909,Umai Sushi Teriyaki,4a96e520f964a520702720e3,Japanese Restaurant,47.608799,-122.340282,Blarney Stone Pub & Restaurant,53275b21498e7dc5f6eb9c56,Pub,47.608225,-122.339326
1273,Eat Seattle,5591a095498e4f08a11b995f,Tour Provider,47.608695,-122.339909,Umai Sushi Teriyaki,4a96e520f964a520702720e3,Japanese Restaurant,47.608799,-122.340282,Hard Rock Cafe Seattle,4a57e9def964a5200fb71fe3,Theme Restaurant,47.609235,-122.339482
1274,Eat Seattle,5591a095498e4f08a11b995f,Tour Provider,47.608695,-122.339909,Umai Sushi Teriyaki,4a96e520f964a520702720e3,Japanese Restaurant,47.608799,-122.340282,The Green Room,42911d00f964a52009241fe3,Bar,47.608570,-122.339622
1275,Eat Seattle,5591a095498e4f08a11b995f,Tour Provider,47.608695,-122.339909,Umai Sushi Teriyaki,4a96e520f964a520702720e3,Japanese Restaurant,47.608799,-122.340282,Kern's Music Shop,52f6dc8d498e81738146ff09,Bar,47.608535,-122.339506


And let's see how many date triples we have

In [12]:
print("There are " + str(len(date_triples)) + " date triples")

There are 1277 date triples


## Part 4: Collect venue data

Compile a list of unique venue IDs. Idenally we'd query each venue to determine price and rating, but we would quickly run up against Foursquare's 50 premium query daily quota. Given the limitation, we'll not be using venue data.

In [13]:
venue_ids = date_triples['e_id']
venue_ids = venue_ids.append(date_triples['r_id'])
venue_ids = venue_ids.append(date_triples['b_id'])
print("There are " + str(len(venue_ids)) + " unique venues")

There are 3831 unique venues


Define a function to get a venue's data from Foursquare. Note this is a premium query and is limited to 50 per day for a developer account.

In [14]:
def getVenueData(v_id):
    
    # create the API request URL
    url = 'https://api.foursquare.com/v2/venues/{}?&client_id={}&client_secret={}&v={}'.format(
            v_id,
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION)
            
    # make the GET request
    response = requests.get(url).json()
    if ("venue" in response["response"]):
            # we have a well formed response, so continue
            results = response["response"]['groups'][0]['items']
    
    print(response)
    #if ("groups" in response["response"]):
    
    return

In [15]:
TEST_VENUE_ID = '41a91500f964a520521e1fe3'
getVenueData(TEST_VENUE_ID)

KeyError: 'groups'

## Part 5: Analyze date triples

How many date triples are there?

In [16]:
 print("There are " + str(len(date_triples)) + " dates available.")

There are 1277 dates available.


What is the most common type of entertainment?

In [17]:
most_common_entertainment = date_triples['e_cat'].describe()['top']
print("The most common entertainment is: " + str(most_common_entertainment))

The most common entertainment is: Comedy Club


What are the restaurant options?

In [18]:
restaurant_options = date_triples['r_cat'].unique()
print("There are " + str(len(restaurant_options)) + " different restaurant options")

There are 44 different restaurant options


How many unique types of date are there?

In [19]:
unique_dates = date_triples[['e_cat', 'r_cat', 'b_cat']].drop_duplicates()
unique_date_types = len(unique_dates)
print("There are " + str(unique_date_types) + " unique dates")

There are 751 unique dates


How many dates are there where each venue is different?

In [20]:
distinct_dates = date_triples.loc[(date_triples['r_id'] != date_triples['b_id'])]
print("There are " + str(len(distinct_dates)) + " distinct dates.")

There are 1265 distinct dates.


## Part 6: Show date triples

We'll now plot date triples on the map to give users the option to explore further. We begin by finding the centroid of each triple's location and adding it to the triple's row in the dataframe.

In [22]:
centroid_cols = ['c_lat', 'c_lng']
centroids = pd.DataFrame(columns = centroid_cols)
for d_ind, d in distinct_dates.iterrows():
    centroid_lat = d[['e_lat', 'r_lat', 'b_lat']].sum()/3
    #centroid_lat = sum(d['e_lat'], d['r_lat'], d['b_lat'])/3
    centroid_lng = d[['e_lng', 'r_lng', 'b_lng']].sum()/3
    #centroid_lng = sum(d['e_lng'], d['r_lng'], d['b_lng'])/3
    centroid = pd.DataFrame([[centroid_lat, centroid_lng]], index=[d_ind], columns=centroid_cols)
    centroids = centroids.append(centroid, ignore_index=True)
    
centroid_dates = distinct_dates.merge(centroids, left_index=True, right_index=True)
print("There are " + str(len(centroid_dates)) + " dates to map")

There are 1253 dates to map


We add color coding to each entertainment option.

In [23]:
e_ids = centroid_dates['e_id'].unique()
colors_array = cm.rainbow(np.linspace(0, 1, len(e_ids)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
e_colors = pd.DataFrame(columns=['e_id','e_color'])
e_colors['e_id'] = e_ids
e_colors['e_color'] = rainbow
e_colors

,e_id,e_color
0,4b8c305df964a5201fc332e3,#8000ff
1,5c033f14e4c459002cd9c82e,#7610ff
2,4cf16e377e0da1cde1949197,#6c1fff
3,4a28ae9cf964a52046951fe3,#622ffe
4,4d7808ce396f3704df2db238,#583efd
5,4a28aedff964a52047951fe3,#4e4dfc
6,4c5b57c29b28d13ab7045470,#445cfb
7,59a989e5f62f2b57ab314c20,#396bf9
8,4ec9ecf26da1d10931751e3f,#3079f7
9,4d0a67c9d4018cfad9ad4728,#2489f5


Finally, we plot all the options on a map.

In [24]:
# create map
map_date_triples = folium.Map(location=[user_region_geo.latitude, user_region_geo.longitude], zoom_start=13)

# add markers to the map
markers_colors = []
for t_ind, t in centroid_dates.iterrows():
    t_color = e_colors[e_colors['e_id'] == t['e_id']]['e_color'].values[0]
    label = folium.Popup(
        "Event: <a target='_blank' href='https://foursquare.com/v/" + str(t['e_id']) + "'>" + str(t['e_name']) + "</a><br>" +
        "Food: <a target='_blank' href='https://foursquare.com/v/" + str(t['r_id']) + "'>" + str(t['r_name']) + "</a><br>" +
        "Drink: <a target='_blank' href='https://foursquare.com/v/" + str(t['b_id']) + "'>" + str(t['b_name']) + "</a>"
        )
    folium.CircleMarker(
        [t['c_lat'], t['c_lng']],
        radius=10,
        popup=label,
        color=t_color,
        fill=True,
        fill_color=t_color,
        fill_opacity=0.4).add_to(map_date_triples)
       
map_date_triples

That's it for the prototype!